In [28]:
import pandas as pd
import numpy as np
import nltk
import re
from textblob import TextBlob
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [29]:
orig_df= pd.read_csv("Stemmed_Data2.csv")
orig_df.drop(['Unnamed: 0'], inplace=True, axis=1)
# print(orig_df.columns)
orig_df.head()

,class,tweet,processed_text,stemmed
0,1,!!! RT @mayasolovely: As a woman you shouldn't...,"['woman', 'complain', 'cleaning', 'house', 'ma...","['woman', 'complain', 'clean', 'hous', 'man', ..."
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,"['boy', 'dats', 'cold', '...', 'tyga', 'dwn', ...","['boy', 'dat', 'cold', '...', 'tyga', 'dwn', '..."
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,"['dawg', 'ever', 'fuck', 'bitch', 'start', 'cr...","['dawg', 'ever', 'fuck', 'bitch', 'start', 'cr..."
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,"['_g_anderson', '_based', 'look', 'like', 'tra...","['_g_anderson', '_base', 'look', 'like', 'tran..."
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,"['shit', 'hear', 'might', 'true', 'might', 'fa...","['shit', 'hear', 'might', 'true', 'might', 'fa..."


In [30]:
orig2= pd.read_csv('Stemmed_Data2.csv')
print(orig2.shape)
orig2.head()

(40290, 5)


,Unnamed: 0,class,tweet,processed_text,stemmed
0,0,1,!!! RT @mayasolovely: As a woman you shouldn't...,"['woman', 'complain', 'cleaning', 'house', 'ma...","['woman', 'complain', 'clean', 'hous', 'man', ..."
1,1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,"['boy', 'dats', 'cold', '...', 'tyga', 'dwn', ...","['boy', 'dat', 'cold', '...', 'tyga', 'dwn', '..."
2,2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,"['dawg', 'ever', 'fuck', 'bitch', 'start', 'cr...","['dawg', 'ever', 'fuck', 'bitch', 'start', 'cr..."
3,3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,"['_g_anderson', '_based', 'look', 'like', 'tra...","['_g_anderson', '_base', 'look', 'like', 'tran..."
4,4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,"['shit', 'hear', 'might', 'true', 'might', 'fa...","['shit', 'hear', 'might', 'true', 'might', 'fa..."


In [31]:
# orig2['tweet']=orig2['tweet'].apply(str)
# orig_df['tweet']=orig_df['tweet'].apply(str)

In [32]:
def splitTags(s):
    l = [i  for i in s.split() if i.startswith("#") ]
    for i in l:
        expanded = " ".join([a for a in re.split('([A-Z][a-z]+)', i[1:]) if a])
        s = s.replace(i,expanded)
    return s

In [33]:
def count_punctuation_and_allcaps(tweet):
    p={'allCaps':0}
    if type(tweet)==str:
        tweet_str= tweet
    else:
        tweet_str=' '.join(tweet)
    temp= re.findall('[A-Z]{2,}', tweet_str)
    p['allCaps']=len(temp)
    
    for i in tweet:
        if i in string.punctuation and i not in ['@','#'] and i not in p:
            p[i]=tweet.count(i)
    return p

In [34]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
#     tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
#     tweet = re.sub(r'^RT[\s]+', '', tweet)
#     tweet= re.sub('[!*(RT)]','',tweet)
    tweet= re.sub("!*\sRT?", ' ', tweet)
    tweet= re.sub("\A!*", '',  tweet)
    tweet= re.sub("\A\sRT\s", ' ',  tweet)
    
    
    tweet = re.sub(r'RT', '', tweet)
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
    x= count_punctuation_and_allcaps(tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = splitTags(tweet)
#     print(tweet)
    # remove hashtags
#     # only removing the hash # sign from the word
#     tweet = re.sub(r'#', '', tweet)
#     tokenize tweets
    
# %%%%%%% changed preserve case parameter to True
    tokenizer = TweetTokenizer(preserve_case=True, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    
#     print(x)
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
#             tweets_clean.append(stem_word)

    return tweets_clean, x

In [35]:
def addPreprocessingCoulmns(df):
    Cols= df.shape[0]
    df['QuestionMarkCount']=[None]* Cols
    df['QuotationMarkCount']=[None]* Cols
    df['ExclamationMarkCount']=[None]* Cols
    df['AllCapitalised']=[None]* Cols
    for i in range(Cols):
        preprocess, l= process_tweet(df['tweet'][i])
        df['processed_text'][i]= preprocess
        df['QuestionMarkCount'][i]= l['?'] if '?' in l else 0
        df['QuotationMarkCount'][i]= l['\''] if '\'' in l else 0
        df['ExclamationMarkCount'][i]=l['!'] if '!' in l else 0
        df['AllCapitalised'][i]= l['allCaps'] if l['allCaps'] else 0
        
    return df

In [36]:
addPreprocessingCoulmns(orig2)
orig2.to_csv('stemmed_semantic_data2.csv')

<ipython-input-35-3e2a2f50e15a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'][i]= preprocess
<ipython-input-35-3e2a2f50e15a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QuestionMarkCount'][i]= l['?'] if '?' in l else 0
<ipython-input-35-3e2a2f50e15a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QuotationMarkCount'][i]= l['\''] if '\'' in l else 0
<ipython-input-35-3e2a2f50e15a>:12: SettingWith

In [37]:
def debug_stepwise_process_tweet(n):
    print(orig2['tweet'][n])
    print('---------------\n')
    s= re.sub("!*\sRT?", ' ', orig2['tweet'][n])
    print(s)
    t= re.sub("\A!*", '',  s)
    print(t)
    r=re.sub("\A\sRT\s", ' ',  t)
    print(r)
    p = re.sub(r'@[A-Za-z0-9]+', '', r)
    print(p)
    q = re.sub(r'https?:\/\/.*[\r\n]*', '', p)
    print(q)
    x,y= process_tweet(orig2['tweet'][n])
    print(x)
    z=count_punctuation_and_allcaps(r)
    print(z)
    
    
debug_stepwise_process_tweet(4)

!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
---------------

  @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
  @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
  @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
  : The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
  : The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
['The', 'shit', 'hear', 'might', 'true', 'might', 'faker', 'bitch', 'told', 'ya', '\ue011']
{'allCaps': 0, ':': 1, '&': 1, ';': 1}


In [38]:
count_punctuation_and_allcaps("FEMIN Dharamshi !!!!!!")

{'allCaps': 1, '!': 6}

In [39]:
def generate_punctuation_rows(tweet):
#     QuestionMarkCount | QuotationMarkCount | ExclamationMarkCount | AllCapitalised
    x, p= process_tweet(tweet) # need to use process tweet to remove redundant exclamation and RT string
#     print(p)
    res=[]
    if '?' in p: 
        res.append(p['?']) 
    else:
        res.append(0)
    if '\'' in p: 
        res.append(p['\'']) 
    else:
        res.append(0)
    if '!' in p: 
        res.append(p['!']) 
    else: 
        res.append(0)
   
    res.append(p['allCaps']) 
    return res

In [40]:
generate_punctuation_rows('She is such a BAD PERSON !!!!!!!')

[0, 0, 7, 2]

### QuestionMarkCount | QuotationMarkCount | ExclamationMarkCount | AllCapitalised
